# PREPARE
You don't need mongodb to run this notebook. You can just use ConversaionBufferMemmory instead. This is for testing purpose only.

To be able to run the test in this notebook, you need some file in `CompassUTD\_secret\` folder. The folder should be in the same directory as this notebook. The files are:

`google_key.json`: - Google API key for Google Cloud Platform. You can get one by making a service account
`search_secrets.json`: - Google Custom Search API key and search engine ID. You can get one by making a custom search engine with these settings:
```
{
    "GOOGLE_SEARCH_API": "REPlACE_WITH_YOUR_KEY",
    "COURSE_SEARCH_ID": "REPlACE_WITH_YOUR_KEY",
    "DEGREE_SEARCH_ID": "REPlACE_WITH_YOUR_KEY",
    "RANDOM_SEARCH_ID": "REPlACE_WITH_YOUR_KEY"
  }
```



In [1]:
import os
import sys
from dotenv import load_dotenv

import random
import string

#append the parent path to sys.path
sys.path.append(os.path.dirname(os.getcwd()))

from CompassUTD import CompassInference
from langchain.memory import ReadOnlySharedMemory, MongoDBChatMessageHistory, ConversationBufferMemory,ConversationBufferWindowMemory


load_dotenv()
agent = CompassInference()



## MongoDB Test

In [2]:
# Use MongoDB
connection_string = f"mongodb+srv://{str(os.getenv('MONGODB_LOGIN'))}@{str(os.getenv('MONGODB_LOCATION'))}"

random_letter_string = "Test_".join([random.choice(string.ascii_letters) for _ in range(11)])

message_history = MongoDBChatMessageHistory(
        connection_string=connection_string, session_id= random_letter_string
    )

In [9]:

memmory_store = ConversationBufferWindowMemory(memory_key="chat_history",chat_memory=message_history, k = 4)

read_only_memory = ReadOnlySharedMemory(memory=memmory_store)


#If you want to make a new chat query with same memory, replace the user_message with next message and run this cell
user_message = "Pretty good"

bot_message = agent.run(user_message = user_message, 
                        read_only_memory= ReadOnlySharedMemory(memory=read_only_memory)
                        )

print(bot_message)

#Save chat history
message_history.add_user_message(user_message)
message_history.add_ai_message(bot_message)


Hi there! I'm CompassUTD, the virtual counselor at The University of Texas at Dallas. How can I help you today?

In [4]:
from rich import print
print(
    message_history.messages
)

[
    HumanMessage(content='Make me a sandwhich', additional_kwargs={}, example=False),
    AIMessage(
        content="I'm sorry, I can't help you with that. I'm a virtual assistant, not a sandwich maker. But I can 
help you find some recipes online. Go Comets!",
        additional_kwargs={},
        example=False
    )
]

In [ ]:
print(
    agent.run(
        user_message = "How are you?",
        read_only_memory = ReadOnlySharedMemory(memory=)
    )
)